# Exploratory Data Analysis (EDA) - Lung Cancer CT Scan Dataset

This notebook explores the chest CT scan dataset used for training a deep learning model
to classify between:
- Normal (No Cancer)
- Adenocarcinoma
- Large-Cell Carcinoma
- Squamous Cell Carcinoma

The goal is to understand the dataset, check class distributions, and visualize sample images
before training.

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import seaborn as sns
from collections import Counter

DATA_DIR = "../dataset"

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, "train"),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, "valid"),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(DATA_DIR, "test"),
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print("Classes:", class_names)

## Class Distribution

We check how balanced the dataset is across the four classes.
Imbalanced datasets can affect training and may require techniques like class weighting or data augmentation.

In [ ]:
# Count labels in train dataset
label_counts = Counter()
for _, labels in train_ds.unbatch():
    label_counts[int(labels.numpy())] += 1

# Plot distribution
sns.barplot(x=[class_names[k] for k in label_counts.keys()], y=label_counts.values())
plt.title("Class Distribution in Training Set")
plt.ylabel("Number of Images")
plt.xlabel("Class")
plt.show()

## Sample Images per Class

Let’s visualize some sample CT scan images from each class to get a sense of what the data looks like.

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Image Shapes

We check the shape of the images and confirm the preprocessing step (resizing).


In [ ]:
for images, labels in train_ds.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)

## Pixel Value Range

Since deep learning models perform better when inputs are normalized,
we confirm the pixel value scaling.

In [ ]:
for images, labels in train_ds.take(1):
    print("Min pixel value:", tf.reduce_min(images).numpy())
    print("Max pixel value:", tf.reduce_max(images).numpy())

## Data Augmentation 

If we plan to use augmentation, let’s preview some transformations.


In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Show augmented images
for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        augmented = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented[0].numpy().astype("uint8"))
        plt.title(class_names[labels[0]])
        plt.axis("off")
    break